In [96]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [451]:
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
cursymb = []
token_index = -1
wrapper()

['W', 0, 'program']
program
['I', 0, 'body']
['R', 4, ';']
['W', 1, 'var']
['I', 1, 'a']
var
['R', 1, ',']
['I', 2, 'b']
['R', 3, ':']
['W', 3, 'integer']
var_type
['R', 4, ';']
['W', 11, 'begin']
['I', 2, 'b']
operator
assignment
['W', 12, ':=']
['C', 0, '1']
expression
term
factor
argument
Argument: 1
['R', 4, ';']
['C', 0, '1']
['R', 4, ';']
['C', 0, '1']
expression checked
['R', 4, ';']
['I', 1, 'a']
operator
assignment
['W', 12, ':=']
['C', 1, '5']
expression
term
factor
argument
Argument: 5
['O', 0, '+']
['C', 1, '5']
['O', 0, '+']
['C', 0, '1']
term
factor
argument
Argument: 1
['O', 3, '/']
['C', 2, '3']
factor
argument
Argument: 3
['O', 2, '*']
['R', 5, '(']
factor
['I', 1, 'a']
expression
term
factor
argument
Argument: a
['O', 0, '+']
['I', 1, 'a']
['O', 0, '+']
['I', 2, 'b']
term
factor
argument
Argument: b
['O', 2, '*']
['R', 5, '(']
factor
['I', 1, 'a']
expression
term
factor
argument
Argument: a
['O', 2, '*']
['I', 1, 'a']
factor
argument
Argument: a
['O', 3, '/']
['I', 2,

'extra ";" before else'

In [447]:
def scan():
    global tokens_chain, token_index, nxtsymb, cursymb
    token_index += 1
    if token_index < len(tokens_chain):
        cursymb = nxtsymb
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [184]:
def antiscan():
    global tokens_chain, token_index, nxtsymb
    token_index -= 1
    if token_index >= 0:
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [185]:
def error(text):
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL)
#     ahah/0

In [186]:
def check(response):
    if len(response):
        error(response)
        return False
    else:
        return True

In [448]:
def wrapper():
    global nxtsymb, cursymb
    scan()
    if nxtsymb[2] == 'program':
        if not check(program()):
            return 'invalid program declaration'
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()   
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    while len(operator()) == 0:
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
    print('END', nxtsymb[2])
    if nxtsymb[2] != 'end.':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return '"'+nxtsymb[2]+'" unexpected'
    

In [210]:
def program():
    global nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [211]:
def var():
    global nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expexted'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        scan()
    if nxtsymb[2] != ':':
        return '":" expected'
    scan()
    if not check(var_type()):
        return 'invalid variable type declaration'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [212]:
def var_type():
    global nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return '"[" expected'
        scan()
        if not check(interval()):
            return 'invalid interval syntax'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not check(interval()):
                return 'invalid interval syntax'
            scan()
        if nxtsymb[2] != ']':
            return '"]" expected'
        scan()
        if nxtsymb[2] != 'of':
            return '"of" expected'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'variable type expected'
        return ''
    else:
        return 'array type expected'
        
            

In [213]:
def interval():
    global nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'constant expected'
    scan()
    if nxtsymb[2] != '..':
        return '".." expected'
    scan()
    if nxtsymb[0] != 'C':
        return 'constant expected'
    return ''

In [433]:
def operator():
    global nxtsymb
    print(Fore.GREEN + 'operator' + Style.RESET_ALL)
    
    if nxtsymb[2] == ';':
        pass
    elif nxtsymb[0] == 'I':
        if not check(assignment()):
            return 'invalid assignment declaration'
    elif nxtsymb[2] == 'if':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'then':
            return '"then" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
        scan()
        if nxtsymb[2] == 'else':
            scan()
            if not check(operator()):
                return 'operator expected'
        else:
            antiscan()
    else:
        return 'not operator'
    return ''

In [431]:
def condition():
    global nxtsymb
    print(Fore.GREEN + 'condition' + Style.RESET_ALL)
    
    if not check(expression()):
        return 'expression expected'
    scan()
    if nxtsymb[2] not in ['=', '>', '<', '>=', '<=', '<>']:
        return 'comparasion operation'
    scan()
    if not check(expression()):
        return 'expression expected'
    return ''

In [395]:
def assignment():
    global nxtsymb
    print(Fore.GREEN + 'assignment' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ':=':
        return '":=" expected'
    scan()
    if not check(expression()):
        return 'invalid expression'
    print(Fore.GREEN + 'expression checked' + Style.RESET_ALL)
    return ''

In [413]:
def expression():
    print(Fore.GREEN + 'expression' + Style.RESET_ALL)
    global nxtsymb
    
#     print('Terms:')
    if not check(term()):
        return 'term expected'
#     print(Fore.BLUE + 'Term:' + Style.RESET_ALL, nxtsymb[2])
    scan()
    while nxtsymb[2] in ['+', '-']:
        scan()
        if not check(term()):
            return 'term expected'
#         print(Fore.BLUE + 'Term:' + Style.RESET_ALL, nxtsymb[2])
        scan()
    else:
        antiscan()
        pass
        
    return ''

In [415]:
def term():
    global nxtsymb
    print(Fore.GREEN + 'term' + Style.RESET_ALL)
    
    if not check(factor()):
        return 'factor expected'
#     print(Fore.BLUE + 'Factor:' + Style.RESET_ALL, nxtsymb[2])
    scan()
    while nxtsymb[2] in ['*', '/']:
        scan()
        if not check(factor()):
            return 'factor expected'
#         print(Fore.BLUE + 'Factor:' + Style.RESET_ALL, nxtsymb[2])
        scan()
    else:
        antiscan()
        pass
#         print('End of factors')
        
    return ''

In [421]:
def factor():
    global nxtsymb
    print(Fore.GREEN + 'factor' + Style.RESET_ALL)
    
    if nxtsymb[2] == '(':
        scan()
        if not check(expression()):
            return 'expression expected'
        scan()
        if nxtsymb[2] != ')':
            return '")" expected'
    else:
        if not check(argument()):
            return 'constant or identifier expected'
#         print('heh')
    return ''

In [401]:
def argument():
    global nxtsymb
    print(Fore.GREEN + 'argument' + Style.RESET_ALL)
    
    if nxtsymb[0] not in ['I', 'C']:
        return 'constant or identifier expected'
    print(Fore.BLUE + 'Argument:' + Style.RESET_ALL, nxtsymb[2])
    return ''